In [2]:
# Importing necessary libraries

import numpy as np

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Suppressing warnings
import warnings
warnings.filterwarnings('ignore')

GPU Availability

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device : {device}')

Using device : cpu


Data Augmentation

In [4]:
imgSize = 224

transformer = transforms.Compose([
    transforms.RandomRotation(degrees = 20),
    
    transforms.Resize(size = (imgSize, imgSize), antialias = True),
    transforms.CenterCrop(size = (imgSize, imgSize)),
    
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

Loading Dataset

In [5]:
path = '/kaggle/input/skin-cancer-malignant-vs-benign/train'
valPath = '/kaggle/input/skin-cancer-malignant-vs-benign/test'

trainData = datasets.ImageFolder(root = path, transform = transformer)
valData = datasets.ImageFolder(root = valPath, transform = transformer)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/skin-cancer-malignant-vs-benign/train'